In [11]:
import pandas as pd 
import os

In [2]:
!pip install selenium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
import pandas as pd 
import os
import re 
import numpy as np


from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='/Users/alexaj/anaconda3/bin/chromedriver')

In [13]:
url = "https://degreeworks.banner.marist.edu/dashboard/dashboard"
driver.get(url)

In [14]:
#driver.implicitly_wait(5)
contFrame = driver.find_element_by_css_selector('html > frameset > frame:nth-child(4)')
driver.switch_to.frame(contFrame)
#frBC = driver.find_element_by_xpath('/html/frameset')
bodyFrame = driver.find_element_by_name('frBody')
driver.switch_to.frame(bodyFrame)

In [15]:
html = driver.page_source
soup = BeautifulSoup(html, "html5lib")

In [6]:
#bodyCode
relevant = soup.find_all("form", attrs={"name": "frmAudit"})

In [151]:
#code for required courses 
reqStrings = soup.find_all("td", attrs={"class": "RuleLabelTitleNeeded"})

reqCourses = []
for reqString in reqStrings:
    reqCourses.append(reqString.text)

coreReqs = [x for x in reqCourses if '3 credits' in x]
#coreReqs
coreReqDf = pd.DataFrame(coreReqs)
coreReqDf.columns = ['Core Requirement']

coreReqDf

,Core Requirement
0,3 credits Ethics/Applied Ethics/Religious Studies


In [150]:
#Code for scraped data from Student View information
studentTable = soup.find("table", attrs={"class": "Inner"})
studentDataTag = studentTable.findAll('td')

studentData = []

for data in studentDataTag:
    #Below one line extracts data 
    studentData.append(data.text)

#studentData
#studentTitle = studentData[::2]
#studentD = studentData[1::2]
#studentTitle
keyVal = dict(zip(studentData[::2], studentData[1::2]))
studentView = pd.DataFrame.from_dict(keyVal, orient='index')
studentView

,0
Student,"Javellana, Alexa"
Level,Undergraduate
ID,20068940
Degree,B.S.
Classification,Junior
College,Computer Science & Mathematics
Advisor,"\nCannistra, Robert M"
Major,Computer Science
Overall GPA,3.476
Minor,Math


In [87]:
#transfer this dataframe into a CSV
filename = 'SerenityStudentInfo.csv'
studentView.to_csv(filename, index=False)

In [16]:
#This code parses the data on DegreeWorks by td class, BlockHeadTitle. 
#Used to correlate the data parsed from BlockHeadSubData involving credits to it's major/minor
headTitle = soup.find_all("td", attrs={"class": "BlockHeadTitle"})
#subTitle
headTitleP = []
for headT in headTitle:
    string = headT.getText()
    headTitleP.append(string)
#headTitleP

creditTitles = []
for headTitle in headTitleP:
    if any(s in headTitle for s in ('Degree', 'Major', 'Minor')):
        creditTitles.append(headTitle)

#titlesDF = pd.DataFrame(creditTitles)
#titlesDF.columns = ['Title']
#titlesDF

In [32]:
semYear = datetime.date.today().year
semMonth = datetime.date.today().month
semSsn = ""

if (semMonth >= 1 and semMonth <= 5):
  semSsn = "Spring"
elif (semMonth >= 6 and semMonth <= 8):
  semSsn = "Summer"
elif (semMonth >= 9 and semMonth <= 12):
  semSsn = "Fall"
elif (semMonth == 1):
  semSsn = "Winter"

semSsn + " " + str(semYear)

'Fall 2018'

In [114]:
currClassHTML = soup.find_all("table", attrs={"class": "xBlocks"})[1]
currClassHTML

# currClasses = pd.read_html(str(currClassHTML))[0]

# currClasses = currClasses.drop([0, 1, 2, 11, 12, 13])
# currClasses = currClasses.drop(columns = [5, 6])
# currClass = pd.DataFrame(currClasses)

# currClass

<table border="0" cellpadding="0" cellspacing="1" class="xBlocks" width="100%">
<tbody><tr><td colspan="20"><table border="0" cellpadding="0" cellspacing="0" class="BlockHeader" width="100%"><tbody><tr>
<td class="BlockHeader" colspan="1" nowrap="" rowspan="2" valign="middle">
                
                      In-progress </td>
<td align="right" width="30%"><table border="0" cellpadding="2" cellspacing="1" class="BlockHeader" width="100%"><tbody><tr>
<td align="right" class="SectionTableSubTitle">Credits
                     Applied: 
                  </td>
<td align="right" class="SectionTableSubData">17</td>
<td class="SectionTableSubTitle">
                     Classes Applied: 
                  </td>
<td align="right" class="SectionTableSubData">7</td>
</tr></tbody></table></td>
</tr></tbody></table></td></tr>
<tr class="CourseAppliedRowWhite">
<td class="ClassesAppliedClasses">BUS 120N</td>
<td class="SectionCourseTitle" width="50%">FINANCIAL LITERACY</td>
<td class="Sectio

In [149]:
currClassHTML = soup.find_all("table", attrs={"class": "xBlocks"})[1]

classTitles = currClassHTML.find_all("td", attrs={"class": "SectionCourseTitle"})
classNames = []

for classes in classTitles:
    classNames.append(classes.text)
    
classNames

['FINANCIAL LITERACY',
 'THEORY OF PROGRAM LANG',
 'COMPUTER ORG & ARCH',
 'OPERATING SYSTEMS',
 'CS PROJECT',
 'CS PROJECT II',
 'TENNIS']

In [80]:
################# STUDENT VIEW SCRAPE #################

def studentInfoScrape(soup):
  studentTable = soup.find("table", attrs={"class": "Inner"})
  studentDataTag = studentTable.findAll('td')

studentData = []

  for stuData in studentDataTag:
      #Below one line extracts data 
      string = stuData.decode_contents(formatter="html")
      studentData.append(string)

  #studentData
  #studentTitle = studentData[::2]
  #studentD = studentData[1::2]
  #studentTitle
  studentKeyVal = dict(zip(studentData[::2], studentData[1::2]))
  studentView = pd.DataFrame.from_dict(studentKeyVal, orient='index')

  studentInfoFile = 'SerenityStudentInfo.csv'
  studentView.to_csv(studentInfoFile, index=False)
#####

studentInfoScrape(soup)

In [155]:
#This code parses the data on DegreeWorks by td class, BlockHeadSubd
#Code parses credits into total and completed arrays

subData = soup.find_all("td", attrs={"class": "BlockHeadSubData"})
#dataNP = array of subData that is not parsed, but converted from obj to str
subDataNP = []
for subD in subData:
    subDataNP.append(subD.text)
    #objToText
    
#dataP = array subData that is/will be parsed
subDataP = []
for subDataN in subDataNP:
    string = subDataN.split("\xa0 ",1)[1]
    if string:
        subDataP.append(string)
#subDataP
    
credits = []

for subData in subDataP:
    if not any(s in subData for s in ('-', '.')):
        credits.append(subData)

totalCredits = []
completedCredits = []
i = 0
while i < len(credits):
    if i % 2 == 0:
        totalCredits.append(credits[i])
        i += 1
    else:
        completedCredits.append(credits[i])
        i += 1

In [156]:
#Combined arrays to a credit progress table 
#creditTitles, completed, total arrays
creditsProgress = np.vstack((creditTitles, completedCredits, totalCredits)).T
creditsProgress = creditsProgress.tolist()
progressDf = pd.DataFrame(creditsProgress,columns=['Title', 'Credits Completed', 'Total Needed'])
progressDf

,Title,Credits Completed,Total Needed
0,Degree in Bachelor of Science,104,120
1,Major in Computer Science,64,67
2,Minor in Math,8,18


In [17]:
ppmDatas = soup.find_all("span", attrs={"class": "item_count"})
ppmNames = soup.find_all("span", attrs={"class": "item_name"})

In [13]:
ppm = []
for ppmData in ppmDatas:
  sep = ' '
  rest = ppmData.text.split(sep, 1)[0]  
  ppm.append(rest)
#ppm

NameError: name 'ppmDatas' is not defined

In [14]:
ppmCountry = []
for ppmName in ppmNames:
  #sep = ' '
  #rest = ppmName.text.split(sep, 1)[0]  
  ppmCountry.append(ppmName.text)    
#ppmCountry

NameError: name 'ppmNames' is not defined

In [201]:
ppmInfo = np.vstack((ppmCountry, ppm)).T
ppmInfo = ppmInfo.tolist()

In [202]:
df = pd.DataFrame(ppmInfo,columns=['By country', 'Page views per minute'])

In [204]:
df

,By country,Page views per minute
0,GB,0.0176
1,US,0.011
2,IE,0.00347
3,DE,0.00288
4,AT,0.00278
5,BR,0.00198
6,SK,0.00169
7,HU,0.00139
8,RU,0.00129
9,VE,0.000794


In [205]:
filename = '0816CountryView.csv'
df.to_csv(filename, index=False)
#os.rename("/Users/alexaj/LastWeekConsolidation/" + filename, "/Users/alexaj/LastWeekConsolidation/AJAXData/" + filename)